# 🛡️ Phishing Email Detection — NLP Module
### DistilBERT Fine-Tuning | Mini Project Phase 3
---
⚠️ **Enable GPU FIRST before running anything!**
```
Runtime → Change runtime type → T4 GPU → Save
```
📂 **Dataset:** `Phishing_Email.csv`
- 18,650 emails
- Columns: `Email Text`, `Email Type`
- Labels: `Safe Email` = 0, `Phishing Email` = 1
---

## ✅ Step 1 — Check GPU & Install Libraries

In [ ]:
import torch

if torch.cuda.is_available():
    print('✅ GPU ready:', torch.cuda.get_device_name(0))
else:
    print('❌ No GPU detected!')
    print('   Fix: Runtime → Change runtime type → T4 GPU → Save')
    print('   Then: Runtime → Restart and run all')

In [ ]:
!pip install transformers scikit-learn seaborn -q
print('✅ Libraries installed!')

## ✅ Step 2 — Upload Dataset

In [ ]:
from google.colab import files
print('📂 File picker will appear below...')
print('   Select your Phishing_Email.csv and wait for upload to complete.')
uploaded = files.upload()
print('\n✅ Uploaded:', list(uploaded.keys()))

## ✅ Step 3 — Load & Explore Data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load — drop the unnamed index column
df = pd.read_csv('Phishing_Email.csv')
df = df.drop(columns=['Unnamed: 0'], errors='ignore')

print('Shape:', df.shape)
print('Columns:', df.columns.tolist())
print('\nLabel distribution:')
print(df['Email Type'].value_counts())
print('\nSample rows:')
display(df.head(3))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 4))

# Class balance
counts = df['Email Type'].value_counts()
colors = ['steelblue', 'tomato']
axes[0].bar(counts.index, counts.values, color=colors, edgecolor='black')
axes[0].set_title('Class Distribution', fontsize=13)
axes[0].set_ylabel('Count')
for i, v in enumerate(counts.values):
    axes[0].text(i, v + 80, str(v), ha='center', fontweight='bold')

# Email length distribution
df['text_length'] = df['Email Text'].fillna('').apply(len)
for label, color in [('Safe Email', 'steelblue'), ('Phishing Email', 'tomato')]:
    axes[1].hist(
        df[df['Email Type'] == label]['text_length'].clip(upper=5000),
        bins=50, alpha=0.6, color=color, label=label
    )
axes[1].set_title('Email Length Distribution', fontsize=13)
axes[1].set_xlabel('Character Count')
axes[1].set_ylabel('Frequency')
axes[1].legend()

plt.tight_layout()
plt.show()

## ✅ Step 4 — Clean & Split Data

In [ ]:
from sklearn.model_selection import train_test_split

# Drop missing and duplicate rows
df = df.dropna(subset=['Email Text', 'Email Type'])
df = df.drop_duplicates(subset='Email Text')
df = df.reset_index(drop=True)

# Convert labels: Phishing Email = 1, Safe Email = 0
df['label'] = (df['Email Type'] == 'Phishing Email').astype(int)

# Truncate very long emails (DistilBERT max is 512 tokens)
df['Email Text'] = df['Email Text'].str[:1500]

print(f'Total emails after cleaning: {len(df)}')
print(f'Safe     (0): {(df["label"]==0).sum()}')
print(f'Phishing (1): {(df["label"]==1).sum()}')

# 80 / 10 / 10 split  — stratified to keep label balance in each split
train_df, temp_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df['label'])
val_df, test_df   = train_test_split(
    temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

print(f'\nTrain: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}')

## ✅ Step 5 — Tokenize with DistilBERT

In [ ]:
from transformers import DistilBertTokenizer

print('Loading DistilBERT tokenizer...')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

MAX_LEN = 256  # 256 tokens is enough for emails; saves GPU memory

def tokenize(texts):
    return tokenizer(
        list(texts),
        padding=True,
        truncation=True,
        max_length=MAX_LEN,
        return_tensors='pt'
    )

print('Tokenizing train set...')
train_enc = tokenize(train_df['Email Text'])
print('Tokenizing val set...')
val_enc   = tokenize(val_df['Email Text'])
print('Tokenizing test set...')
test_enc  = tokenize(test_df['Email Text'])
print('\n✅ Tokenization complete!')

## ✅ Step 6 — Create PyTorch Dataset

In [ ]:
import torch
from torch.utils.data import Dataset

class EmailDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.reset_index(drop=True).tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

train_dataset = EmailDataset(train_enc, train_df['label'])
val_dataset   = EmailDataset(val_enc,   val_df['label'])
test_dataset  = EmailDataset(test_enc,  test_df['label'])

print(f'✅ Train: {len(train_dataset)} | Val: {len(val_dataset)} | Test: {len(test_dataset)}')

## ✅ Step 7 — Load DistilBERT & Fine-Tune
> ⏱️ **~15–25 minutes on T4 GPU. Do NOT close the browser tab!**
> 
> 💡 If you get `CUDA out of memory` error → change `per_device_train_batch_size=16` to `8`

In [ ]:
from transformers import DistilBertForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('Loading DistilBERT model...')
model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=2  # 0 = Safe Email, 1 = Phishing Email
)
model.to(device)
print(f'✅ Model loaded on: {device}')

def compute_metrics(pred):
    labels = pred.label_ids
    preds  = pred.predictions.argmax(-1)
    p, r, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy':  round(acc, 4),
        'f1':        round(f1, 4),
        'precision': round(p, 4),
        'recall':    round(r, 4)
    }

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,  # ← change to 8 if CUDA out of memory
    per_device_eval_batch_size=32,
    warmup_steps=100,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    logging_steps=50,
    report_to='none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

print('🚀 Training started...')
print('You will see Epoch 1/3 → 2/3 → 3/3 progress bars below.')
print('Expected time: ~15–25 mins on T4 GPU')
print('-' * 50)
trainer.train()
print('\n✅ Training complete!')

## ✅ Step 8 — Evaluate on Test Set

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print('Running predictions on test set...')
predictions = trainer.predict(test_dataset)
preds  = predictions.predictions.argmax(-1)
labels = predictions.label_ids

print('\n' + '='*50)
print('        CLASSIFICATION REPORT')
print('='*50)
print(classification_report(
    labels, preds,
    target_names=['Safe Email', 'Phishing Email']
))

In [ ]:
cm = confusion_matrix(labels, preds)

plt.figure(figsize=(7, 5))
sns.heatmap(
    cm, annot=True, fmt='d', cmap='Blues',
    xticklabels=['Safe Email', 'Phishing Email'],
    yticklabels=['Safe Email', 'Phishing Email'],
    annot_kws={'size': 14}
)
plt.title('Confusion Matrix — Test Set', fontsize=14)
plt.ylabel('Actual Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.show()

tn, fp, fn, tp = cm.ravel()
print(f'✅ Phishing correctly caught:       {tp}')
print(f'✅ Safe emails correctly identified: {tn}')
print(f'⚠️  Phishing missed (false negative): {fn}')
print(f'⚠️  Safe wrongly flagged (false pos): {fp}')

## ✅ Step 9 — Live Prediction Function
> 🎯 Use this for your mini project demo!

In [ ]:
import torch.nn.functional as F

def predict_email(text):
    """Predict if an email is Safe or Phishing."""
    model.eval()
    inputs = tokenizer(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=256,
        padding=True
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    probs      = F.softmax(outputs.logits, dim=-1)
    predicted  = probs.argmax().item()
    confidence = probs.max().item()
    result     = '🚨 PHISHING EMAIL' if predicted == 1 else '✅ SAFE EMAIL'

    short = text[:80] + '...' if len(text) > 80 else text
    print(f'Input:      "{short}"')
    print(f'Prediction: {result}')
    print(f'Confidence: {confidence:.2%}')
    print(f'Safe: {probs[0][0].item():.2%}  |  Phishing: {probs[0][1].item():.2%}')
    print('-' * 65)

# ── Demo predictions ─────────────────────────────────────────────
print('='*65)
print('          LIVE PHISHING DETECTION DEMO')
print('='*65)

predict_email(
    "Your account has been suspended. Click here immediately to verify "
    "your identity and avoid permanent closure of your account."
)

predict_email(
    "Hi team, please find the Q3 meeting notes attached. "
    "Let me know if you have any questions."
)

predict_email(
    "Congratulations! You have been selected for a $1000 Amazon gift card. "
    "Claim your prize now before it expires!"
)

predict_email(
    "Dear customer, your PayPal account has been limited. "
    "Please update your billing information to restore full access."
)

predict_email(
    "Hey, are you joining us for lunch today? "
    "We are thinking of trying the new place on 5th street."
)

In [ ]:
# ── Try YOUR OWN email text here ─────────────────────────────────
my_email = """Type or paste any email text here and run this cell!"""

predict_email(my_email)

## ✅ Step 10 — Save & Download the Model

In [ ]:
# Save model + tokenizer
model.save_pretrained('./phishing_detector_model')
tokenizer.save_pretrained('./phishing_detector_model')
print('✅ Model saved to ./phishing_detector_model/')

# Zip and download
!zip -r phishing_detector_model.zip ./phishing_detector_model

from google.colab import files
files.download('phishing_detector_model.zip')
print('✅ Download started — check your browser downloads folder!')

---
## 🎉 Phase 3 Complete!

| Step | Task | Status |
|------|------|--------|
| 1 | GPU check + install libraries | ✅ |
| 2 | Upload `Phishing_Email.csv` | ✅ |
| 3 | Load & explore (18,650 emails) | ✅ |
| 4 | Clean + 80/10/10 split | ✅ |
| 5 | DistilBERT tokenization | ✅ |
| 6 | PyTorch dataset creation | ✅ |
| 7 | Fine-tune 3 epochs (~20 mins) | ✅ |
| 8 | Evaluate + confusion matrix | ✅ |
| 9 | Live prediction demo | ✅ |
| 10 | Save & download model | ✅ |

**Expected results:**
- Accuracy: ~97–98%
- F1 Score: ~0.96–0.98

---